#### This assignment may be worked individually or in pairs. 
#### Enter your name(s) here: Cameron Courtney, Viren Velacheri


In [ ]:
# Cameron Courtney, Viren Velacheri

# Assignment 2: KNN and NB classifiers

In this assignment you'll implement the K-Nearest Neighbors (KNN) and Naive Bayes (NB)  classifiers to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the same Diabetic Retinopathy data set which was used in the previous assignment on decision trees.

In [ ]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import time

Q0. Read the data from a CSV file and store it in a Pandas dataframe.

In [ ]:
def get_data(filename):
    data = []
    #your code goes here
    # Reads csv file as a dataframe
    data = pd.read_csv(filename, header = None)
    return data

You may use the following function to print a confusion matrix:

In [ ]:
def print_confusion_matrix(TP, FN, FP, TN):
    table_data = [[TP,FN],[FP,TN]]
    df = pd.DataFrame(table_data, columns =['Predicted 1','Predicted 0'])
    df = df.rename(index={0: 'Actual 1', 1: 'Actual 0'})
    display(df)


## Part 1: K Nearest Neighbor (KNN) Classifier

Q1. Normalize the data so that each feature value lies between `[0-1]`.

In class, we talked about why scaling the data is critical to KNN. We also talked about how data scaling should be done *inside the cross validataion loop*. This means that the scaling parameters should be based on the **training set only**, in order to prevent data leakage. Then the test data will need to be scaled, using the parameters found on the **training** data.

Fill in the function to take in a training dataset and a test dataset and normalize them correctly. Return the normalized datasets.

Caution: Return NEW datasets that have been normalized - do not normalize the datasets in-place, so that this can be run numerous times without altering the original data or normalizing already normalized data.

Hint: When using dataframes, you can do this without a loop!

In [ ]:
def normalize_data(train, test):
    # your code goes here
    # Datasets are normalized and returned
    train_norm = (train - train.min()) / (train.max() - train.min())
    test_norm = (test - train.min()) / (train.max() - train.min())
    return train_norm, test_norm

Q2. The distance calculation method is central to the KNN algorithm. In this assignment you'll be using the Euclidean distance. 

Implement a function that takes in one data point (as a list), and the training data (as a dataframe), and calculates the Euclidian distance from the single data point to each of the data points in the training data.

You may return these however you want (or may add them to the dataframe?).

Hint: 
For KNN, the distance calculations are the most time-consuming part of the algorithm. Even though computing Euclidian distance seems like a simple, and therefore quick, calculation, running it thousands of times, inside of a nested 5-fold cross-validation for example, can cause this algorithm to take a very long time to run, depending on your implementation. 

Remember, you almost never need to loop a Dataframe! Pandas DataFrames have been specifically optimized for fast operations on large datasets, by [vectorizing](https://www.quantifisolutions.com/vectorization-part-2-why-and-what) calculations across all rows at once.

If you use a DataFrame, you should not write a loop to calculate each of the Euclidian distances one at a time. Look at [this post](https://stackoverflow.com/questions/46908388/find-euclidean-distance-from-a-point-to-rows-in-pandas-dataframe?rq=1) for more info.

Caution: Be careful not to use the label in your distance calculation.

In [ ]:
def get_distances(point, df):
    # your code goes here 
    # Euclidean distances are calculated and returned for respective point
    df['distances'] = df.iloc[:, :19].sub(np.array(point[0:19])).pow(2).sum(1).pow(0.5)
    return df['distances']

Q3. Build your KNN classifier.

This function takes in a training set (as a dataframe), a test set (as a dataframe), and a k to use, and classifies all data points in the test set, using the data in the training set and the given k.

It should return the predicted labels for the test set as a list.

Caution: Remember to normalize your data before doing distance calculations.

In [ ]:
def run_knn(train_set, test_set, k):
    # your code goes here
    train_norm, test_norm = normalize_data(train_set, test_set)
    predictions = []
    # Simple for loop where euclidean distances are calculated for all the 
    # points in test set. Training set is sorted based on this and then 
    # the top k rows labels are selected. Based on what the majority label is, 
    # that is the prediction label made and appended to list of predictions to
    # be returned at the end
    for row in range(len(test_norm.index)):
      train_norm_copy = train_norm.copy(deep=True)
      distances = get_distances(list(test_norm.iloc[row, :]), train_norm_copy)
      train_norm_copy.sort_values(by='distances', inplace=True)
      labels = train_norm_copy.iloc[0:k, 19]
      if labels.sum() >= k/2:
        predictions.append(1)
      else:
        predictions.append(0)
    return predictions

Q4. Find the best value of k for this data. 

Try k ranging from 1 to 10 (odds only). For each k value, use a 5-fold cross validation to evaluate the accuracy with that k. In each fold of CV, divide your data into a training set and a validation set. Print out the best value of k and the accuracy achieved with that value. Return the best value of k. If there is a tie for best k, use the lowest of the k values.

Hint: This is the *inner* loop of a nested cross validation.

In [ ]:
def find_best_k(data):
    # your code goes here
    best_k = None
    max_accuracy = -1
    # i goes through all the possible odd values from 1 through 10
    for i in range(1, 10, 2):
      accuracies = []
      # 5 fold cross validation loop happening here
      for j in range(5):
        lower_bound = int((j / 5) * len(data.index))
        upper_bound = int(((j + 1) / 5) * len(data.index))
        train_set = data[0: lower_bound].append(data[upper_bound: len(data.index)])
        test_set = data[lower_bound: upper_bound]
        predictions = run_knn(train_set, test_set, i)
        prediction_index = 0
        num_correct = 0
        # Accuracies calculated for each fold
        for index in range(len(test_set.index)):
          if predictions[index] == test_set.iloc[index, 19]:
            num_correct += 1
        accuracies.append(num_correct/len(test_set.index))
      # Average accuracy calculated at end of 5 folds and if greater than the 
      # current max accuracy, it is replaced and the associated k value is 
      # recorded
      average_accuracy = sum(accuracies) / len(accuracies)
      if average_accuracy > max_accuracy:
        max_accuracy = average_accuracy
        best_k = i
    print("The best k value is " + str(best_k) + " and the accuracy achieved is " 
          + str(max_accuracy * 100)) 
    return best_k

Q5. Now measure the accuracy of your classifier using 5-fold cross validation. 

In each fold of this CV, divide your data into a training set and a test set. The training set should get sent through your code for Q4, resulting in a value of k to use. Using that k, calculate an accuracy on the test set. You will average the accuracy over all 5 folds to obtain the final accuracy measurement. 

Print the accuracy, the confusion matrix, and the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [ ]:
# read in data
data = get_data('messidor_features.txt')
start_time = time.time()
# your code goes here
accuracies = []
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
# five fold cross validation loop
for i in range(5):
  lower_bound = int((i / 5) * len(data.index))
  upper_bound = int(((i + 1) / 5) * len(data.index))
  train_set = data[0: lower_bound].append(data[upper_bound: len(data.index)])
  test_set = data[lower_bound: upper_bound]
  best_k = find_best_k(train_set)
  predictions = run_knn(train_set, test_set, best_k)
  num_correct = 0
  for index in range(len(test_set.index)):
    if predictions[index] == test_set.iloc[index, 19]:
      num_correct += 1
      if predictions[index] == 1:
        true_positives += 1
      else:
        true_negatives += 1
    else:
      if predictions[index] == 1:
        false_positives += 1
      else:
        false_negatives += 1
  print("The accuracy is ", str(num_correct/len(test_set.index) * 100))
  accuracies.append(num_correct/len(test_set.index))

average_accuracy = sum(accuracies) / len(accuracies)
print("The overall accuracy is ", str(average_accuracy * 100))
print_confusion_matrix(true_positives, false_negatives, false_positives, true_negatives)
precision = true_positives / (true_positives + false_positives)
print("Precision is " + str(precision))
recall = true_positives / (false_negatives + true_positives)
print("Recall is " + str(recall))
end_time = time.time()
print('\nTotal time:', end_time - start_time)

The best k value is 7 and the accuracy achieved is 64.06345475910692
The accuracy is  61.73913043478261
The best k value is 5 and the accuracy achieved is 63.5229142185664
The accuracy is  62.17391304347826
The best k value is 9 and the accuracy achieved is 62.319623971797874
The accuracy is  64.34782608695652
The best k value is 9 and the accuracy achieved is 65.25616921269096
The accuracy is  59.56521739130435
The best k value is 5 and the accuracy achieved is 62.39130434782608
The accuracy is  62.33766233766234
The overall accuracy is  62.03274985883681


Predicted 1  Predicted 0
Actual 1          363          248
Actual 0          189          351

Precision is 0.657608695652174
Recall is 0.5941080196399345

Total time: 74.15861368179321


## Part 2: Naive Bayes Classifier

Naive Bayes (NB) classifier is a simple probabilistic classifier that is based on applying the Bayes' theorem and assumes a strong (naive) independence between features. The Diabetic Retinopathy data set contains both categorical and continuous features. Dealing with categorical features is straightforward. Continuous attributes, on the other hand, are more interesting to handle. Most commonly, this is done by assuming normal probability distribution over the feature values or by binning the attribute values in a fixed number of bins. In this assignment you'll be implementing the binning approach. 

Q6. Implement bin_features which takes in a dataset as a dataframe and for each continuous attribute, constructs 3 equal sized bins. For example, feature 2 ranges from `[1 - 132]`, so the bin widths will be (132-1)/3 = 43.667, and the 3 bins that you'll construct will be `[1 - 44.667]`, `(44.667 - 88.334]`, `(88.334 - 132]`.

The result of this function can be whatever/however you want. (We will not directly test this function with an autograder.)

Note: You may hard-code the indexes of the continuous vs categorical features if you want. 

Caution: Don't bin the labels.

In [ ]:
def bin_features(data):
    # your code goes here
    # Loop through only the continuous attributes/features
    for i in range(2, 18):
      max_val = data.iloc[:,i].max()
      min_val = data.iloc[:,i].min()
      bin_width = (max_val - min_val)/3
      # initialized new column for assigned bin values to essentially nulls
      # at the start
      data['Bin for ' + str(i)] = pd.NaT
      bin_list = []
      # a simple loop iterating through each row where associated bin is 
      # calculated and appended to bin list where new column is to be set to
      for j in range(len(data.index)):
        if data.iloc[j, i] < min_val or (data.iloc[j,i] >= min_val and data.iloc[j, i] <= (min_val + bin_width)):
          bin_list.append(1)
        elif data.iloc[j,i] > (min_val + bin_width) and data.iloc[j, i] <= (min_val + 2 * bin_width):
          bin_list.append(2)
        else:
          bin_list.append(3)
      data['Bin for ' + str(i)] = bin_list
    return data

Q7. Given a dataset, calculate the prior probability of each class and return them.

Hint: With a dataframe, you can do this without looping. 

In [ ]:
def get_priors(data):
    # your code goes here
    # return as a tuple (prior of class 0, prior of class 1)
    total_num = len(data.index)
    # simple filtering used for getting number of zero and one class labels
    # to be used for calculating these prior probabilities
    num_zero = len(data[data.iloc[:,19] == 0].index)
    num_one = len(data[data.iloc[:,19] == 1].index)
    prob_zero = num_zero/total_num
    prob_one = num_one/total_num
    return (prob_zero, prob_one) 

Q8. Given a training data set, a feature index, and a feature value of the record to classify, return the class conditional probabilities for both classes. Remember to apply Laplace smoothing, if necessary!

In [ ]:
# Helper function created to help with determining what bin feature value
# belongs to
def feature_val_classification(data, feat_idx, feat_value):
  max_val = data.iloc[:,feat_idx].max()
  min_val = data.iloc[:,feat_idx].min()
  bin_width = (max_val - min_val)/3
  if feat_value < min_val or (feat_value >= min_val and feat_value <= min_val + bin_width):
    return 1
  elif feat_value > (min_val + bin_width) and feat_value <= (min_val + 2 * bin_width):
    return 2
  else:
    return 3

In [ ]:
def get_class_condit_prob(train_data, feat_idx, feat_value):
    # your code goes here
    # return as a tuple (p(feat|class0), p(feat|class1))
    # Get associated dataframes for zero and one labels respectively
    zero_df = train_data[train_data.iloc[:,19] == 0]
    one_df = train_data[train_data.iloc[:,19] == 1]
    binary_index = [0, 1, 18]
    feature_class = feat_value
    # used in case one of the calculated probabilities turns out to be zero
    # and laplace smoothing has to take place
    possible_denom_val = 2
    index1 = zero_df.iloc[:,feat_idx]
    index2 = one_df.iloc[:,feat_idx]
    # feature index continuous so use respective column in data frame that has
    # the bin values for the respective feature index.
    if feat_idx not in binary_index:
      feature_class = feature_val_classification(train_data, feat_idx, feat_value)
      possible_denom_val = 3
      index1 = zero_df['Bin for ' + str(feat_idx)]
      index2 = one_df['Bin for ' + str(feat_idx)]
    zero_prob = len(zero_df[index1 == feature_class].index) / len(zero_df.index)
    one_prob = len(one_df[index2 == feature_class].index) / len(one_df.index)
    # Laplace smoothing takes place if either probability is zero
    if zero_prob == 0 or one_prob == 0:
      zero_prob = (len(zero_df[index1 == feature_class].index) + 1) / (len(zero_df.index) + possible_denom_val)
      one_prob = (len(one_df[index2 == feature_class].index) + 1) / (len(one_df.index) + possible_denom_val)
    return (zero_prob, one_prob) 

Q9. Build your Naive Bayes classifier.

This function takes in a training set (as a dataframe) and a test set (as a dataframe), and classifies all data points in the test set, using the data in the training set.

It should return the predicted labels for the test set as a list.

If there is a tie, predict label 1 (positive for disease).

Remember to bin the features based on the **training set only**. If the test set were used to determine bin ranges, this would be data leakage! If a test record has a feature value less than the first bin, consider it to be in the first bin. If a test record has a feature value greater than the last bin, consider it to be in the last bin. 

Caution: Do not use the labels in your prediction calculation.

In [ ]:
def nb_classify(train_set, test_set):
    # your code goes here
    # features in training set are binned
    bin_set = bin_features(train_set)
    prob_zero, prob_one = get_priors(train_set)
    predictions = []
    # simply loop through for all rows. For each row,
    # go through associated feature and get probabilities to be multiplied.
    # At end the probability that is higher, the respective prediction is made
    # and added to the list of predictions to be returned at end
    for row in range(len(test_set.index)):
      curr_prob_zero = prob_zero
      curr_prob_one = prob_one
      for col in range(19):
        zero_prob, one_prob = get_class_condit_prob(bin_set, col, test_set.iloc[row, col])
        curr_prob_zero *= zero_prob
        curr_prob_one *= one_prob
      if curr_prob_one >= curr_prob_zero:
        predictions.append(1)
      else:
        predictions.append(0)
    return predictions

Q10. Now measure the accuracy of your classifier using 5-fold cross validation. 

In each fold of this CV, divide your data into a training set and a test set. The training & test sets should get sent through your code for Q9. You will average the accuracy over all 5 folds to obtain the final accuracy measurement. 

Print the accuracy, the confusion matrix, and the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [ ]:
# read in data
data = get_data('messidor_features.txt')
start_time = time.time()

# your code goes here
accuracies = []
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
# Five fold cross validation loop
for i in range(5):
  lower_bound = int((i / 5) * len(data.index))
  upper_bound = int(((i + 1) / 5) * len(data.index))
  train_set = data[0: lower_bound].append(data[upper_bound: len(data.index)])
  test_set = data[lower_bound: upper_bound]
  predictions = nb_classify(train_set, test_set)
  num_correct = 0
  for index in range(len(test_set.index)):
    if predictions[index] == test_set.iloc[index, 19]:
      num_correct += 1
      if predictions[index] == 1:
        true_positives += 1
      else:
        true_negatives += 1
    else:
      if predictions[index] == 1:
        false_positives += 1
      else:
        false_negatives += 1
  print("The accuracy is ", str(num_correct/len(test_set.index) * 100))
  accuracies.append(num_correct/len(test_set.index))

average_accuracy = sum(accuracies) / len(accuracies)
print("The overall accuracy is ", str(average_accuracy * 100))
print_confusion_matrix(true_positives, false_negatives, false_positives, true_negatives)
precision = true_positives / (true_positives + false_positives)
print("Precision is " + str(precision))
recall = true_positives / (false_negatives + true_positives)
print("Recall is " + str(recall))
end_time = time.time()
print('\nTotal time:', end_time - start_time)

The accuracy is  63.91304347826087
The accuracy is  61.73913043478261
The accuracy is  54.78260869565217
The accuracy is  53.91304347826087
The accuracy is  61.038961038961034
The overall accuracy is  59.07735742518352


Predicted 1  Predicted 0
Actual 1          310          301
Actual 0          170          370

Precision is 0.6458333333333334
Recall is 0.5073649754500819

Total time: 62.56976795196533
